# TP 4 - Deep Learning
# ECM 3A - UE IA - Option Digitale - 2018-2019

In [0]:
from __future__ import print_function

In [1]:
### Pour utiliser sur google colab

import os

from google.colab import drive
drive.mount('drive')

!mkdir -p drive -v
#!google-drive-ocamlfuse drive

cwd = os.getcwd()

#### Changez le chemin ci-dessous vers votre repertoire dans votre googledrive
##############################################################################
monchemin = 'drive/My Drive/IAAA/2018/DeepLearningCourse/Optim/Hyperas/'
##############################################################################

dir_path  = os.path.join(cwd, monchemin)
dirs = os.listdir(dir_path)
os.chdir(dir_path)

###  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


## 1. Comparaisons des algorithmes de gradient 

###  Convnet pour la classification de Mnist

#### Basics

In [2]:
## Imports

import keras
from keras.engine import Layer
from keras import backend as K
import numpy as np
from keras.legacy import interfaces

from keras import metrics

from keras import optimizers
from keras import regularizers
from keras.layers import Input, BatchNormalization, concatenate, Reshape, Conv2DTranspose, Activation

from keras.layers import Input, Concatenate
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D, MaxPooling2D
from keras.datasets import mnist
from keras.optimizers import Adam, Adagrad
from keras import backend as K
from keras import initializers
from keras.utils import np_utils
from keras import regularizers

from keras import backend as K


from keras.datasets import mnist

Using TensorFlow backend.


In [0]:


def init_datas():
	# input image dimensions
	img_rows, img_cols = 28, 28
	nb_classes = 10

	##### Chargement des donnees

	# the data, shuffled and split between train and test sets
	(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

	
	if 'tf' == 'th':
		X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
		X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
		input_shape = (1, img_rows, img_cols)
	else:
		X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
		X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
		input_shape = (img_rows, img_cols, 1)


	X_train = X_train.astype('float32')
	X_test = X_test.astype('float32')
	X_train = X_train /255
	X_test = X_test /255
	print('X_train shape:', X_train.shape)
	print('X_test shape:', X_test.shape)
	print(X_train.shape[0], 'train samples')
	print(X_test.shape[0], 'test samples')

	# convert class vectors to binary class matrices
	y_train = keras.utils.to_categorical(Y_train, nb_classes)
	y_test = keras.utils.to_categorical(Y_test, nb_classes)
	print('y_train shape:', y_train.shape)
	print('y_test shape:', y_test.shape)

	return X_train, y_train, X_test, y_test

In [4]:
input_shape = (28, 28, 1)

nb_classes =10 
X_train, Y_train, X_test, Y_test = init_datas()

11493376/11490434 [==============================] - 0s 0us/step
X_train shape: (60000, 28, 28, 1)
X_test shape: (10000, 28, 28, 1)
60000 train samples
10000 test samples
y_train shape: (60000, 10)
y_test shape: (10000, 10)


In [5]:
print(X_train.shape)

(60000, 28, 28, 1)


In [0]:
### Added 4 10 2018
def create_discri(opt= 'Adam'):
    d_input = Input(shape=(28, 28, 1) ,name="input")
    x = Conv2D(64, kernel_size=(3, 3), activation='relu') (d_input)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    d_prediction = Dense(nb_classes, activation='softmax')(x)
    discriminatorC = Model(input = d_input, output = d_prediction)
    discriminatorC.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[metrics.categorical_accuracy])
    return discriminatorC


### To do 

Comparez les routines d'optimisation Adam, Adagrad, Adadelta etc en utiisant dfférents learning rates poru l'apprentissage des modèles défnis ci-dessus

Vous tracerez deux figures, l'une pour l'accuracy, l'autre pour le loss, calculés sur le test set.

Dans chaque figure vous ploitterez la quantité (Accuracy ou Loss) en fonction du learning rate en incluant une courbe par routine d'optimisation.

## Dropout

In [0]:
### Added 4 10 2018
def create_discri(opt= 'Adam', DO_rate1=0.5, DO_rate2=0.5):
    d_input = Input(shape=(28, 28, 1) ,name="input")
    x = Conv2D(64, kernel_size=(3, 3), activation='relu') (d_input)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(DO_rate1)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(DO_rate2)(x)
    d_prediction = Dense(nb_classes, activation='softmax')(x)
    discriminatorC = Model(input = d_input, output = d_prediction)
    discriminatorC.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[metrics.categorical_accuracy])
    return discriminatorC


In [8]:
from keras.optimizers import Adam, Adagrad, Adadelta

Perfs={}
T_DO1 = [0., 0.25, 0.5]
T_DO2 = [0., 0.25, 0.5]

Optimizers = ["Adam", "Adagrad", "Adadelta"]
for opt in Optimizers:
    Perfs[opt]={}
    for do1 in T_DO1:
      for do2 in T_DO2:
        o = eval(opt)()
        D = create_discri(opt=o, DO_rate1 = do1 , DO_rate2 = do2)
        D.fit(X_train, Y_train, epochs=1)
        Perfs[opt][str(do1), str(do2)]=D.evaluate(X_test,Y_test)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()





Epoch 1/1





10000/10000 [==============================] - 1s 110us/step

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/1
10000/10000 [==============================] - 1s 115us/step


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/1
19392/60000 [========>.....................] - ETA: 14s - loss: 0.2781 - categorical_accuracy: 0.9162

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-e3c0637713e5>", line 14, in <module>
    D.fit(X_train, Y_train, epochs=1)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1178, in fit
    validation_freq=validation_freq)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_arrays.py", line 204, in fit_loop
    outs = fit_function(ins_batch)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2979, in __call__
    return self._call(inputs)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2937, in _call
    fetched = self._callable_fn(*array_vals)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py", line 1472, in __call__
    run_metadata_ptr)
KeyboardInterrup

KeyboardInterrupt: ignored

In [0]:


print (Perfs["Adam"])

print (Perfs["Adagrad"])

print (Perfs["Adadelta"])



## Batch Normalization

### To do 

1. Rajoutez une couche de BatchNormalization et expliquez le nombre de paramètres trainable et not trainable


2. Calculez les statistiques sur les activations moyennes (et les variances)  dans les différentes couches. Vous devez apprendre un réseau avec BN et un autre sans, puis faire passer les données d'apprentissage dans le réseau et extraire les prédiction sur les différentes couches cachées, puis afficher les valeurs moyennes d'activation sur les différentes couches 

NB: Le comportement de la couche de BN est plus visible avec des réseaux profonds non convolutionnels


## Optimisation des hypermaramètres

Quel que soit le problème, quelles que soient vos données, il existe un certain nombre d'hyper paramètres à régler pour apprndre un bon modèle (architceture du modèle, paramètres de l'algorithme d'optimisation etc).

C'est l'objet du gridsearch, qui peut être exhaustif ou pas.

Avant de vous lancer dans un gridsearch faites une estumation du temps que cela prendra.

### Gridsearch


En utilisant un embedding des modèles dans sklearn et le gridsearchCV de sklearn

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [0]:


def create_discri_BN(optimizer='rmsprop', nb_filters1= 64,  nb_filters2= 32, fact = 'relu', nb_hid = 128, do_rate= 0.5, BN=False):
    d_input = Input(shape=(28, 28, 1) ,name="input")
    x = Conv2D(nb_filters1, kernel_size=(3, 3), activation='relu') (d_input)
    x = Conv2D(nb_filters2, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    if BN :
      x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(nb_hid)(x)
    x = Activation(fact)(x)
    x = Dropout(do_rate)(x)
    d_prediction = Dense(nb_classes, activation='softmax')(x)
    discriminatorC = Model(input = d_input, output = d_prediction)
    discriminatorC.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])
    return discriminatorC
  

In [0]:

model = KerasClassifier(build_fn=create_discri_BN)

optimizers = ['rmsprop']
epochs = [10]
V_nb_hid = [64, 128]
DO_rate=[0, 0.25, 0.5]

param_grid = dict(optimizer=optimizers,  nb_hid =V_nb_hid, do_rate= DO_rate)

print (param_grid)

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train, epochs = 1, verbose=2)
# summarize results                                                                                                                                           
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Si vous utilisez un cluster de calcul, il est souvent plus simple 
- De lancer un job par jeu d'hyperparametres
=> Boucles énumérant toutes les conbinaisons : Un job lancé à chaque fois



### Optimisation Hyperas 
Cf. https://conference.scipy.org/proceedings/scipy2013/pdfs/bergstra_hyperopt.pdf
Stratégie d'exploration partielle et informée de l'espace de recherche 

La méthode explore une partie de l'espace de recherche en construisant au fur et à mesure une mesure d'intérêt des différets paramètres à faire varier (voir l'article cité cidessus).

Nb: Pour utiliser hyperas avec google colab votre notebook doit etre sauvegardé sur google colab et vous devez spécifier son chemin (voir cellule suivante).


In [0]:


!pip install hyperas
!pip install hyperopt

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

def init_datas():
    # input image dimensions
    img_rows, img_cols = 28, 28
    nb_classes = 10

    ##### Chargement des donnees

    # the data, shuffled and split between train and test sets
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    #X_train = X_train.reshape((X_train.shape[0],784))
    ###X_train = X_train.reshape(X_train.shape[0], 784) 
    ###X_test = X_test.reshape(X_test.shape[0], 784)
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
  
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train = X_train /255
    X_test = X_test /255
    print('X_train shape:', X_train.shape)
    print('X_test shape:', X_test.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = keras.utils.to_categorical(Y_train, nb_classes)
    Y_test = keras.utils.to_categorical(Y_test, nb_classes)
    print('y_train shape:', Y_train.shape)
    print('y_test shape:', Y_test.shape)

    return X_train, Y_train, X_test, Y_test



def create_model(X_train, Y_train, X_test, Y_test):
   
    """Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again."""
 

    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Flatten())
    model.add(Dense({{choice([64, 128, 256])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    # If we choose 'four', add an additional fourth layer

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})

    model.fit(X_train, Y_train,
              batch_size={{choice([64, 128])}},
              epochs=1,
              verbose=2,
              validation_data=(X_test, Y_test))
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


init_datas()  

In [0]:


best_run, best_model = optim.minimize(model=create_model,
                                      data=init_datas2,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(), 
                                      notebook_name='TP4_DL_2018_OptimDNN_Etus')
X_train, Y_train, X_test, Y_test = init_datas2()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

## To do : Régularisation et taille de l'ensemble d'apprentissage

- Trouver la meilleure stratégie d'apprentissage pour une architcteure neuronale donnée, sur les données Mnist et pour des jeux de données limités. Vous utiliserez plusieurs tirages random pour chaque taille et calculerez des résultats moyennés. Vous utiliserez deux tailles de base d'apprentissage.
--  1000 données d'apprentissage en tout (environ 100 par classe)
-- 5000 données d'apprentissage en tout (environ 500) ? 

- Vous discuterez suit à vos résultats de l'intérêt de la régularisation dans le cas de données limitées et de l'Intérêt des différentes techniques de regularisation(dropout, pénalisation L1, L2)
